In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
# Set column names
ekko_col_names = ['data', 'sample start time']
ekkolodd_df = pd.read_csv('raw/ekkolodd.txt', names=ekko_col_names, header=None)

In [3]:
# Split after confidence
ekkolodd_df['data'] = ekkolodd_df['data'].str.split(r'\%')

In [4]:
# separate to rows
ekkolodd_df = ekkolodd_df.explode('data')

In [5]:
# split distance and confidence
ekkolodd_df['data'] = ekkolodd_df['data'].str.split(r'\t')

In [6]:
# drop all data values that are not distance:xx, confidence:xx
ekkolodd_df = ekkolodd_df[ekkolodd_df.apply(lambda x: True if len(x.data) == 2 else False, axis=1)]

In [7]:
# split distance and confidence to columns
ekkolodd_df['distance'], ekkolodd_df['confidence'] = zip(*ekkolodd_df['data'])

In [8]:
# only keep the integer and set the column type
ekkolodd_df['confidence'] = ekkolodd_df['confidence'].str.replace(r'[^0-9]', '').astype(int)
ekkolodd_df['distance'] = ekkolodd_df['distance'].str.replace(r'[^0-9]', '').astype(int)

C:\Users\Ferdi\AppData\Local\Temp\ipykernel_15076\4058848018.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  ekkolodd_df['confidence'] = ekkolodd_df['confidence'].str.replace(r'[^0-9]', '').astype(int)
C:\Users\Ferdi\AppData\Local\Temp\ipykernel_15076\4058848018.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  ekkolodd_df['distance'] = ekkolodd_df['distance'].str.replace(r'[^0-9]', '').astype(int)


In [9]:
# remove the data column as we now have two new columns
ekkolodd_df.drop(['data'], axis=1, inplace=True)

In [10]:
# paste the latest sample time value to the NaN cells
ekkolodd_df['sample start time'] = ekkolodd_df['sample start time'].fillna(ekkolodd_df['sample start time'].dropna().iloc[-1])

In [12]:
# save to csv file
ekkolodd_df.to_csv('data/ekkolodd.csv', index=False)

In [ ]:
ekkolodd_df

,sample start time,distance,confidence
1,09 Aug 2022 13:15:16 +0000,6751,100
1,09 Aug 2022 13:15:16 +0000,6751,100
1,09 Aug 2022 13:15:16 +0000,6751,100
1,09 Aug 2022 13:15:16 +0000,6751,100
1,09 Aug 2022 13:15:16 +0000,6751,100
...,...,...,...
77,25 Aug 2022 10:28:29 +0000,6227,0
77,25 Aug 2022 10:28:29 +0000,6227,0
77,25 Aug 2022 10:28:29 +0000,6722,0
77,25 Aug 2022 10:28:29 +0000,6722,0


In [117]:
with open('gyroskop.txt') as f:
    gyroskop_raw = f.read().splitlines()

gyroskop = []
for line in gyroskop_raw:
    gyroskop.append(re.split('\), \(', line))

gyroskop

[['[]'],
 ['[]'],
 ['[]'],
 ['[]'],
 ['[]'],
 ['[]'],
 ["[('$ISDPT,0000.000,M,000.0000,B', '0', '0'",
  "'$ISDPT,-000.264,M,000.9906,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9963,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9963,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9963,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9963,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9963,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9963,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9962,B', '0', '0'",
  "'$ISDPT,-000.208,M,000.9962,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,000.9961,B', '0', '0'",
  "'$ISDPT,-000.210,M,00

In [112]:
for val in gyroskop[6]:
    print(re.split(', ', re.sub('\(','',val)))

["['$ISDPT,0000.000,M,000.0000,B'", "'0'", "'0'"]
["'$ISDPT,-000.264,M,000.9906,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9963,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9963,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9963,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9963,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9963,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9963,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9962,B'", "'0'", "'0'"]
["'$ISDPT,-000.208,M,000.9962,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,M,000.9961,B'", "'0'", "'0'"]
["'$ISDPT,-000.210,

In [88]:
with open('trykksensor.txt') as f:
    trykksensor_raw = f.read().splitlines()

trykksensor = []
for line in trykksensor_raw:
    trykksensor.append(re.split("\\\\r\\\\n", re.sub("'b'",'',line)))

In [89]:
trykksensor

[['sampling started at :Mon, 25 Oct 2021 15:45:20 +0000 '],
 [" b''sampling started at :Mon, 25 Oct 2021 15:48:05 +0000 "],
 [" b''sampling started at :Mon, 25 Oct 2021 15:50:12 +0000 "],
 [" b''sampling started at :Mon, 25 Oct 2021 15:50:12 +0000 "],
 [" b''sampling started at :Mon, 25 Oct 2021 15:52:27 +0000 "],
 [' sampling started at :Mon, 25 Oct 2021 16:17:15 +0000 '],
 [" b'\\x00\\xff$ISDPT,0000.000,M,000.0000,B,00.00,C*38",
  '$ISHPR,005.2,+08.1,+177.3*6E',
  '$ISDPT,-000.264,M,000.9906,B,106.42,C*12',
  '$ISHPR,304.2,-01.4,+178.9*63',
  '$ISDPT,-000.208,M,000.9963,B,50.39,C*25',
  '$ISHPR,297.6,-02.0,+178.9*6B',
  '$ISDPT,-000.208,M,000.9963,B,31.08,C*20',
  '$ISHPR,296.4,-02.1,+178.9*69',
  '$ISDPT,-000.208,M,000.9963,B,24.38,C*27',
  '$ISHPR,296.1,-02.1,+178.9*6C',
  '$ISDPT,-000.208,M,000.9963,B,22.06,C*2C',
  '$ISHPR,296.0,-02.0,+178.9*6C',
  '$ISDPT,-000.208,M,000.9963,B,21.25,C*2E',
  '$ISHPR,296.0,-01.9,+179.0*6E',
  '$ISDPT,-000.208,M,000.9963,B,20.98,C*29',
  '$ISHPR,2